James Caldwell, UVA IRA, Feb 2026 <br>
IPEDS ACTS reporting <br> 
A quick script for loading the data files (Ugrad admission, Grad admission, Ugrad completers, Grad completers) into the 7 years' worth of ACTS template files.

In [3]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
parent_folder = os.getenv('parent_folder') 

ugrad_adm = pd.read_excel(os.path.join(parent_folder, 'undergraduate_admissions.xlsx'))
grad_adm = pd.read_excel(os.path.join(parent_folder, 'graduate_admissions.xlsx'))
ugrad_completer = pd.read_excel(os.path.join(parent_folder, 'undergraduate_completers.xlsx'))
grad_completer = pd.read_excel(os.path.join(parent_folder, 'graduate_completers.xlsx'))


In [5]:
template_folder = os.getenv('template_folder') 

year_dict = {
    'ACTS_Template_AY2019-20.xlsx': 1920,
    'ACTS_Template_AY2020-21.xlsx': 2021,
    'ACTS_Template_AY2021-22.xlsx': 2122,
    'ACTS_Template_AY2022-23.xlsx': 2223,
    'ACTS_Template_AY2023-24.xlsx': 2324,
    'ACTS_Template_AY2024-25.xlsx': 2425,
    'ACTS_Template_AY2025-26.xlsx': 2526
}

# This copies the empty template files from the backup folder 
import shutil
import os
from pathlib import Path
src = Path(template_folder+r'\backups')
dst = Path(template_folder)
dst.mkdir(parents=True, exist_ok=True)

for file in src.iterdir():
    if file.is_file():
        shutil.copy2(file, dst / file.name)  # copy2 preserves metadata

In [ ]:
for excel_name, year in year_dict.items():

    ugrad_adm_sub = ugrad_adm[ugrad_adm['reporting_year'] == year].copy()
    ugrad_adm_sub.drop(columns=['sex_imputed','Student System ID','reporting_year'], inplace=True)

    grad_adm_sub = grad_adm[grad_adm['reporting_year'] == year].copy()
    grad_adm_sub.drop(columns=['sex_imputed','Student System ID','reporting_year'], inplace=True)

    ugrad_completer_sub = ugrad_completer[ugrad_completer['reporting_year'] == year].copy()
    ugrad_completer_sub.drop(columns=['sex_imputed','reporting_year'], inplace=True)

    grad_completer_sub = grad_completer[grad_completer['reporting_year'] == year].copy()
    grad_completer_sub.drop(columns=['sex_imputed','reporting_year'], inplace=True)
    
    # ex: 1920 -> 19-20
    excel_year = str(year)[0:2]+ '-' + str(year)[2:4]

    excel_dict = {
        'Template_UGAdm'+excel_year: ugrad_adm_sub,
        'Template_GRAdm'+excel_year: grad_adm_sub,
        'Template_UGComp'+excel_year: ugrad_completer_sub,
        'Template_GRComp'+excel_year: grad_completer_sub
}

    for sheet_name, sheet_df in excel_dict.items():
        if 'Comp' in sheet_name and '25-26' in excel_name:
            continue # skip this year completers since data isn't available yet

        file_path = template_folder + '\\' + excel_name

        # read ONLY the header row (row 4 in Excel)
        existing_cols = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=3,     # row 4 in Excel
            nrows=0       # only read header, no data
        ).columns.tolist()

        # keep only matching columns (exact name match)
        filtered_df = sheet_df[existing_cols]

        with pd.ExcelWriter(
            file_path,
            engine="openpyxl",
            mode="a",                 # open existing file
            if_sheet_exists="overlay" # write onto existing sheet
        ) as writer:
            filtered_df.to_excel(
                writer,
                sheet_name=sheet_name,
                startrow=4, 
                header=False,
                index=False
            )
